# Capstone - Delivery-3
This dataset is part of the [Farming Systems Project](https://www.ars.usda.gov/northeast-area/beltsville-md-barc/beltsville-agricultural-research-center/sustainable-agricultural-systems-laboratory/docs/farming-systems-project/) at USDA, Beltsville MD.  This data is not available online on the USDA
 website but can be found on my [GitHub](https://github.com/mmtokay/DATA606/tree/master/datasets).


The data is split in two files, one that contains crop information and other with weather data.

Crop file:
* Crop - wheat, corn or soybean           
* GrowingSeason - year crop was cultivated 
* SystemName - crop management (traditional: NT and CT; organic: Org2, Org3 and Org6')    
* GrainYield - grain yield measured in kg/ha     
* PlantingDate - date seeds were planted  
* HarvestDate - date crop was harvested


Weather file:
* Year 
* Julian Day 
* Month
* Day
* Date
* avgtTempC - average temperature in C
* maxTempC - maximum temperature in C
* minTempC - minimum temperature in C
* maxHumPct - maximum humidity in %
* minHumPct - minimum humidity in %
* avgRadWm-2 - average radiation in w/m2
* meanWindMs-1 - mean wind in m/s
* PrecipitationMm - precipitation/snow melt in mm

# Feature Engineering


In [1]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import warnings
import time

from datetime import datetime, timedelta
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler, Normalizer, MinMaxScaler, StandardScaler, Binarizer
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.utils import shuffle
from time import time

warnings.simplefilter(action='ignore', category=FutureWarning)

# Crop Data
Import crop data file.

In [2]:
data = pd.read_csv('FSPGrainYieldsV3Clean.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Data columns (total 6 columns):
Crop             1113 non-null object
GrowingSeason    1113 non-null int64
SystemName       1113 non-null object
GrainYield       1113 non-null float64
PlantingDate     1113 non-null object
HarvestDate      1042 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 52.2+ KB


In [3]:
# Convert from object to float64
data['GrainYield'] = pd.to_numeric(data.GrainYield, errors='coerce')

# Convert PlantingDate and HarvestDate from object to date
data['PlantingDate'] = pd.to_datetime(data.PlantingDate)
data['HarvestDate'] = pd.to_datetime(data.HarvestDate)

print("\nLet's check if there is any data missing on the dataset.\n")
data.isna().sum()


Let's check if there is any data missing on the dataset.



Crop              0
GrowingSeason     0
SystemName        0
GrainYield        0
PlantingDate      0
HarvestDate      71
dtype: int64

In [4]:
# Calculate duration between PlantingDate and HarvestDate
data['weekDuration'] = data['HarvestDate'] - data['PlantingDate']
data['weekDuration'] = data['weekDuration']/np.timedelta64(1,'W')
print('\nCheck unique values for Crop, GrowingSeason and SystemName columns.\n')
print("Crop", data.Crop.unique())
print("\nGrowing Season", data.GrowingSeason.unique())
print("\nCrop Management Type", data.SystemName.unique())
print("\nThere are duplicate values for SystemManagement because column values are case sensitive. I will convert SystemName column to uppercase.")
data['SystemName'] = data['SystemName'].str.upper()
print("\nCrop Management Type", data.SystemName.unique())


Check unique values for Crop, GrowingSeason and SystemName columns.

Crop ['CRN' 'SOY' 'WHT']

Growing Season [1996 1997 1998 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2016]

Crop Management Type ['NT' 'CT' 'Org2' 'Org3' 'Org6' 'ORG2' 'ORG3' 'ORG6']

There are duplicate values for SystemManagement because column values are case sensitive. I will convert SystemName column to uppercase.

Crop Management Type ['NT' 'CT' 'ORG2' 'ORG3' 'ORG6']


In [5]:
# 1 for conventional
# 0 for organic
data['SystemNameType'] = ((data.SystemName == "NT") | (data.SystemName == "CT")).map({True:'1', False:'0'})
# Drop SystemName column
data.drop('SystemName', axis=1, inplace=True)
data.head()

,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
0,CRN,1996,10488.0,1996-05-23,1996-10-17,21.0,1
1,CRN,1996,9722.5,1996-05-23,1996-10-17,21.0,1
2,CRN,1996,10008.0,1996-05-23,1996-10-17,21.0,1
3,CRN,1996,8928.8,1996-05-23,1996-10-17,21.0,1
4,CRN,1996,10985.3,1996-05-23,1996-10-17,21.0,1


I will separate the data by crop: corn, soybean and wheat and I will display basic statistics for each crop.

# Corn dataset - Statistics

In [6]:
data_corn = data.loc[data['Crop'] == "CRN"]
data_corn.reset_index(inplace = True)
data_corn.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,390.000000,390,390.000000,390.000000,390,370,370.000000,390
unique,NaN,1,NaN,NaN,43,64,NaN,2
top,NaN,CRN,NaN,NaN,2009-06-29 00:00:00,2008-10-10 00:00:00,NaN,0
freq,NaN,390,NaN,NaN,16,20,NaN,234
first,NaN,NaN,NaN,NaN,1996-05-23 00:00:00,1996-10-17 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2016-06-09 00:00:00,2016-10-18 00:00:00,NaN,NaN
mean,194.500000,NaN,2006.376923,5746.695385,NaN,NaN,21.583398,NaN
std,112.727548,NaN,5.932697,3350.745309,NaN,NaN,2.521325,NaN
min,0.000000,NaN,1996.000000,71.900000,NaN,NaN,16.857143,NaN
25%,97.250000,NaN,2002.000000,2811.000000,NaN,NaN,19.428571,NaN


# Soybean dataset - Statistics

In [7]:
data_soy = data.loc[data['Crop'] == "SOY"]
data_soy.reset_index(inplace = True)
data_soy.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,500.000000,500,500.000000,500.000000,500,465,465.000000,500
unique,NaN,1,NaN,NaN,49,49,NaN,2
top,NaN,SOY,NaN,NaN,2001-05-30 00:00:00,2013-10-29 00:00:00,NaN,1
freq,NaN,500,NaN,NaN,20,20,NaN,273
first,NaN,NaN,NaN,NaN,1996-05-23 00:00:00,1996-10-04 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2016-07-14 00:00:00,2016-10-27 00:00:00,NaN,NaN
mean,639.500000,NaN,2006.768000,2647.311400,NaN,NaN,21.113978,NaN
std,144.481833,NaN,5.767556,1156.686365,NaN,NaN,2.868903,NaN
min,390.000000,NaN,1996.000000,165.200000,NaN,NaN,15.000000,NaN
25%,514.750000,NaN,2002.000000,1888.100000,NaN,NaN,19.142857,NaN


# Wheat dataset - Statistics

In [8]:
data_wheat = data.loc[data['Crop'] == "WHT"]
data_wheat.reset_index(inplace = True)
data_wheat.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,223.000000,223,223.000000,223.000000,223,207,207.000000,223
unique,NaN,1,NaN,NaN,19,15,NaN,2
top,NaN,WHT,NaN,NaN,2001-10-26 00:00:00,2015-07-12 00:00:00,NaN,1
freq,NaN,223,NaN,NaN,16,16,NaN,115
first,NaN,NaN,NaN,NaN,1996-11-04 00:00:00,1997-07-06 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2015-11-17 00:00:00,2016-07-11 00:00:00,NaN,NaN
mean,1001.000000,NaN,2007.578475,4278.827354,NaN,NaN,35.621808,NaN
std,64.518731,NaN,6.094000,1064.753320,NaN,NaN,1.901822,NaN
min,890.000000,NaN,1997.000000,1403.500000,NaN,NaN,31.428571,NaN
25%,945.500000,NaN,2001.500000,3671.850000,NaN,NaN,34.857143,NaN


# Week Duration For Each Crop
I will use the minimum week duration to generate weather features:

corn = 16

soybeans = 15

wheat = 31


# Weather Data

Import weather data.

In [9]:
weather_data = pd.read_csv('FSPWeather1996-2019V2.csv')
weather_data['Date'] = pd.to_datetime(weather_data.Date)
print(weather_data.describe(include="all"))
print("\nLet's check if there is any data missing on the dataset.\n")
print(weather_data.isna().sum())

               Year    JulianDay        Month          Day  \
count   8763.000000  8763.000000  8763.000000  8763.000000   
unique          NaN          NaN          NaN          NaN   
top             NaN          NaN          NaN          NaN   
freq            NaN          NaN          NaN          NaN   
first           NaN          NaN          NaN          NaN   
last            NaN          NaN          NaN          NaN   
mean    2007.498916   183.092320     6.522994    15.729773   
std        6.921633   105.423565     3.449147     8.799551   
min     1996.000000     1.000000     1.000000     1.000000   
25%     2001.500000    92.000000     4.000000     8.000000   
50%     2007.000000   183.000000     7.000000    16.000000   
75%     2013.000000   274.000000    10.000000    23.000000   
max     2019.000000   366.000000    12.000000    31.000000   

                       Date    avgtTempC     maxTempC     minTempC  \
count                  8763  8728.000000  8763.000000  8763.0

In [10]:
weather_data.drop(['Year','JulianDay','Month',' Day','avgRadWm-2'], axis=1, inplace=True)
print(weather_data.describe(include="all"))

                       Date    avgtTempC     maxTempC     minTempC  \
count                  8763  8728.000000  8763.000000  8763.000000   
unique                 8763          NaN          NaN          NaN   
top     2007-09-10 00:00:00          NaN          NaN          NaN   
freq                      1          NaN          NaN          NaN   
first   1996-01-01 00:00:00          NaN          NaN          NaN   
last    2019-12-31 00:00:00          NaN          NaN          NaN   
mean                    NaN    13.059551    18.700260     7.228509   
std                     NaN     9.363676    10.073274     9.275069   
min                     NaN   -13.600000    -9.570000   -20.100000   
25%                     NaN     5.300000    10.600000    -0.500000   
50%                     NaN    13.500000    19.800000     7.062000   
75%                     NaN    21.492350    27.500000    15.500000   
max                     NaN    31.800000    39.700000    26.070000   

          maxHumPct

My next step is to work on feature engineering, I will combine crop mangement type in two categories traditional and organic.  After I determine the average number of weeks that takes for each crop to mature I will calculate weather variables weekly average starting from planting date.

I will also calculate growing degree days (GDD) that "are used to estimate the growth and development of plants and insects during the growing season. The basic concept is that development will only occur if the temperature exceeds some minimum development threshold, or base temperature (TBASE). The base temperatures are determined experimentally and are different for each organism". [1]

GDD formula for corn and soybean:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 10

GDD formula wheat:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 4.4

In [11]:
def calcGDD(df,startDate,endDate,factor):
    gdd = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        gdd = gdd + (((j['maxTempC']+j['minTempC'])/2)-factor)
    return gdd

def calcAverage(df,startDate,endDate,var):
    sum = 0
    avg = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        sum = sum + j[var]
    if sum > 0:
        avg = sum/(i+1)
    return avg

def calcMax(df,startDate,endDate,var):
    val = []
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        val.append(j[var])
    maxVal = max(val)
    return maxVal

def calcMin(df,startDate,endDate,var):
    val = []
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        val.append(j[var])
    minVal = min(val)
    return minVal

def calcSum(df,startDate,endDate,var):
    sum = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        sum = sum + j[var]
    return sum

def createFeaturesMatrix(cropData,weatherData,numWeeks,GDDFactor):
    master_tp = list()
    colName = ()
    i = 0
    j = 0
    for i, j in cropData.iterrows():
        if (i == 0):
            startDate = j['PlantingDate']
        #start calculating date ranges to aggregate weather data for 16 weeks starting from plantingDate
        new_tp = ()
        for w in range(numWeeks):
            temp_tuple = ()
            beginWeek = j['PlantingDate'] + timedelta(days=7)*w
            endWeek = j['PlantingDate'] + timedelta(days=7)*(w+1)
            if(w==(numWeeks-1)):
                temp_tuple = (calcAverage(weather_data,beginWeek,endWeek,'avgtTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxTempC'),\
                              calcMin(weather_data,beginWeek,endWeek,'minTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxHumPct'),\
                              calcMin(weather_data,beginWeek,endWeek,'minHumPct'),\
                              calcAverage(weather_data,beginWeek,endWeek,'meanWindMs-1'),\
                              calcSum(weather_data,beginWeek,endWeek,'PrecipitationMm'),\
                              calcGDD(weather_data,startDate,endWeek,GDDFactor),\
                              j['SystemNameType'],j['GrainYield'])
                if (i == 0):
                    colName = colName + ('avgTemp'+str(w+1),'maxTemp'+str(w+1),'minTemp'+str(w+1),\
                                         'maxHum'+str(w+1),'minHum'+str(w+1),'meanWind'+str(w+1),\
                                         'Precip'+str(w+1),'GDD','SystemNameType','GrainYield')
            else:
                temp_tuple = (calcAverage(weather_data,beginWeek,endWeek,'avgtTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxTempC'),\
                              calcMin(weather_data,beginWeek,endWeek,'minTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxHumPct'),\
                              calcMin(weather_data,beginWeek,endWeek,'minHumPct'),\
                              calcAverage(weather_data,beginWeek,endWeek,'meanWindMs-1'),\
                              calcSum(weather_data,beginWeek,endWeek,'PrecipitationMm'))
                if (i == 0):
                    colName = colName + ('avgTemp'+str(w+1),'maxTemp'+str(w+1),'minTemp'+str(w+1),\
                                         'maxHum'+str(w+1),'minHum'+str(w+1),'meanWind'+str(w+1),\
                                         'Precip'+str(w+1))
            new_tp = new_tp + temp_tuple
        #print(new_tp)
        master_tp.append(new_tp)

    new_df = pd.DataFrame(list(master_tp),columns = colName)
    return(new_df)

# Corn

In [12]:
data_corn.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

/glob/intel-python/versions/2019u1/intelpython3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
new_df16 = createFeaturesMatrix(data_corn,weather_data,16,10)
new_df16.to_csv(r'cornFeatures16w.csv', index = False, header=True)

In [14]:
new_df15 = createFeaturesMatrix(data_corn,weather_data,15,10)
new_df15.to_csv(r'cornFeatures15w.csv', index = False, header=True)

In [15]:
new_df14 = createFeaturesMatrix(data_corn,weather_data,14,10)
new_df14.to_csv(r'cornFeatures14w.csv', index = False, header=True)

# Soybean

In [16]:
data_soy.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

/glob/intel-python/versions/2019u1/intelpython3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
new_soy_df15 = createFeaturesMatrix(data_soy,weather_data,15,10)
new_soy_df15.to_csv(r'soyFeatures15w.csv', index = False, header=True)

In [18]:
new_soy_df14 = createFeaturesMatrix(data_soy,weather_data,14,10)
new_soy_df14.to_csv(r'soyFeatures14w.csv', index = False, header=True)

In [19]:
new_soy_df13 = createFeaturesMatrix(data_soy,weather_data,13,10)
new_soy_df13.to_csv(r'soyFeatures13w.csv', index = False, header=True)

# Wheat

In [20]:
data_wheat.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

/glob/intel-python/versions/2019u1/intelpython3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [21]:
new_wheat_df31 = createFeaturesMatrix(data_wheat,weather_data,31,4.4)
new_wheat_df31.to_csv(r'wheatFeatures31w.csv', index = False, header=True)

In [22]:
new_wheat_df30 = createFeaturesMatrix(data_wheat,weather_data,30,4.4)
new_wheat_df30.to_csv(r'wheatFeatures30w.csv', index = False, header=True)

In [23]:
new_wheat_df29 = createFeaturesMatrix(data_wheat,weather_data,29,4.4)
new_wheat_df29.to_csv(r'wheatFeatures29w.csv', index = False, header=True)